In [15]:
import pandas as pd
from marginal.main import estimar_marginal
from thing import condicional


df = pd.read_csv("weather.csv")
df

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [2]:
def foo(df,colNames, values):
    l = []
    for i in range(len(colNames)):
        l.append(df[colNames[i]] == values[i])
        
#     return df.loc()
#     print(l)
    
    x = l[0]
    for i in range(1,len(l)):
        x = x&l[i]
        
    return df.loc[x]

newDf = foo(df,["temperature"], ["hot"])
newDf

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
3,overcast,hot,normal,False,yes
9,sunny,hot,high,False,no
10,sunny,hot,high,True,no


In [3]:
def findVariables(df, col):
    l = []
    for elem in df[col]:
    	if elem not in l:
    		l.append(elem)
    return l

x = findVariables(newDf, "outlook")
x

['overcast', 'sunny']

In [4]:
def getCardinality(df, col):
    return len(findVariables(df,col))

y = getCardinality(newDf, "outlook")
y

2

In [5]:
# Desde aqui importa <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

m = [[0,1,1,1,0],
     [1,0,0,1,1],
     [0,0,0,1,1],
     [0,0,0,0,0],
     [1,1,1,1,0]]

def getParents(g,df):
    fathers = []
    res = []
    colNames = list(df.columns)
    
    for col in range(len(g)):
        aux = []
        for row in range(len(g)):
            if(m[col][row] == 1):
                aux.append(colNames[row])
        fathers.append(aux)

    for i in range(len(g)):
        yield [colNames[i],fathers[i]]
    
    #return res

for i in getParents(m,df):
    print(i)

['outlook', ['temperature', 'humidity', 'windy']]
['temperature', ['outlook', 'windy', 'play']]
['humidity', ['windy', 'play']]
['windy', []]
['play', ['outlook', 'temperature', 'humidity', 'windy']]


In [6]:
class rb:
    def __init__(self,g, df,a):
        self.dag = g
        self.margs = [] # => tienes que correr calc() para llenarlo
        self.conds = []
        self.alpha = a
        self.df = df
    
    def calc(self):
        for elem in getParents(self.dag,self.df):
            if(len(elem[1])== 0):
                self.margs.append([elem,estimar_marginal(df,elem[0],self.alpha)])
            else:
                self.conds.append([elem,condicional(df,elem[0],elem[1],self.alpha)])

In [13]:
def inferencia(df,query,colNames, values, rb):
    mgs = []
    cds = []
    accum = 1
    colNames = list(df.columns)
    
    
    for i in rb.conds:
        idxs = []
        for e in i[0]:
            if e in colNames:
                idxs.append(colNames.index(e))
            else:
                dictValues = i[1].values
                
                cds.append(max(dictValues))
        vals = []
        for idx in idxs:
            vals.append(values[idx])
        
        cds.append(i[1][vals])
        
    for i in rb.margs: # {0:0.5,1:0,5}
        idxs = []
        for e in i[0]:
            if e in colNames:
                idxs.append(colNames.index(e))
            else:
                dictValues = i[1].values
                mgs.append(max(dictValues))
        vals = []
        for idx in idxs:
            vals.append(values[idx])
            
            mgs.append(i[1][vals])
    
    for i in range(len(mgs)):
        accum *= mgs[i]
        
    for i in range(len(cds)):
        accum *= cds[i]
    return accum

In [8]:
x = rb(m,df,1)
x.calc()

14
36
{('overcast', 'hot', 'high', False): 0.3466666666666667, ('overcast', 'hot', 'high', True): 0.26, ('overcast', 'hot', 'normal', False): 0.52, ('overcast', 'hot', 'normal', True): 0.52, ('overcast', 'cool', 'high', False): 0.52, ('overcast', 'cool', 'high', True): 0.52, ('overcast', 'cool', 'normal', False): 0.17333333333333334, ('overcast', 'cool', 'normal', True): 0.3466666666666667, ('overcast', 'mild', 'high', False): 0.17333333333333334, ('overcast', 'mild', 'high', True): 0.3466666666666667, ('overcast', 'mild', 'normal', False): 0.26, ('overcast', 'mild', 'normal', True): 0.26, ('rainy', 'hot', 'high', False): 0.17333333333333334, ('rainy', 'hot', 'high', True): 0.26, ('rainy', 'hot', 'normal', False): 0.26, ('rainy', 'hot', 'normal', True): 0.52, ('rainy', 'cool', 'high', False): 0.52, ('rainy', 'cool', 'high', True): 0.52, ('rainy', 'cool', 'normal', False): 0.3466666666666667, ('rainy', 'cool', 'normal', True): 0.3466666666666667, ('rainy', 'mild', 'high', False): 0.3466

In [16]:
print(inferencia(df,"A",["B"],["1"],x))

AttributeError: 'NoneType' object has no attribute 'values'